## 기본 합성곱 신경망 구현

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## 모델 정의

In [3]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        # TODO
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        self.sequence = list()
        self.sequence.append(conv2d(16,(3,3), padding='same', activation='relu')) # 28*28*16
        self.sequence.append(conv2d(16,(3,3), padding='same', activation='relu')) # 28*28*16
        self.sequence.append(maxpool((2,2))) # 14*14*16
        self.sequence.append(conv2d(32,(3,3), padding='same', activation='relu')) # 14*14*32
        self.sequence.append(conv2d(32,(3,3), padding='same', activation='relu')) # 14*14*32
        self.sequence.append(maxpool((2,2))) # 7*7*32
        self.sequence.append(conv2d(64,(3,3), padding='same', activation='relu')) # 7*7*64
        self.sequence.append(conv2d(64,(3,3), padding='same', activation='relu')) # 7*7*64
        self.sequence.append(tf.keras.layers.Flatten()) #  1568
        self.sequence.append(tf.keras.layers.Dense(2048, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))
        
    def call(self, x, training=False, mask=None):
        # TODO
        for layer in self.sequence:
            x = layer(x)
        return x

## 학습, 테스트 루프 정의

In [8]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [5]:
# TODO
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # float64로 인식 -> float32(나중에 문제X)

# x_train : (NUM_SAMPLE, 28, 28) -> (NUM_SAMPLE, 28, 28, 1)

x_train = x_train[..., tf.newaxis].astype(np.float32) 
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [9]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.11228369176387787, Accuracy: 96.48833465576172, Test Loss: 0.04003530368208885, Test Accuracy: 98.58999633789062
Epoch 2, Loss: 0.04464925080537796, Accuracy: 98.65333557128906, Test Loss: 0.03672342747449875, Test Accuracy: 98.94000244140625
Epoch 3, Loss: 0.03208991885185242, Accuracy: 99.0566635131836, Test Loss: 0.030166903510689735, Test Accuracy: 99.15999603271484
Epoch 4, Loss: 0.02728166989982128, Accuracy: 99.15166473388672, Test Loss: 0.043363139033317566, Test Accuracy: 98.80999755859375
Epoch 5, Loss: 0.023431148380041122, Accuracy: 99.3116683959961, Test Loss: 0.034200455993413925, Test Accuracy: 99.12999725341797
Epoch 6, Loss: 0.019534699618816376, Accuracy: 99.36833190917969, Test Loss: 0.03516232594847679, Test Accuracy: 99.06999969482422
Epoch 7, Loss: 0.01734294928610325, Accuracy: 99.4816665649414, Test Loss: 0.03631116822361946, Test Accuracy: 99.12999725341797
Epoch 8, Loss: 0.01647549867630005, Accuracy: 99.51166534423828, Test Loss: 0.0316097699